In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 1. Install and Import Dependecies

In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests 
from bs4 import BeautifulSoup
import re
import warnings
warnings.filterwarnings("ignore")

### 2. Instantiate the Model 

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

### 3. Encode and Calculate Sentiment

In [6]:
tokens = tokenizer.encode('I hated this, absolutely worse', return_tensors = 'pt')

In [7]:
tokens

tensor([[  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 84705,   102]])

In [8]:
tokenizer.decode(tokens[0]) #you can decode the tokens too

'[CLS] i hated this, absolutely worse [SEP]'

In [9]:
result = model(tokens)
result
#logits represents the probablity of that particular class being the sentiment 

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.1303,  2.3340, -0.2313, -2.8576, -2.6756]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [10]:
#grab the logists and convert which is useful for us
torch.argmax(result.logits)

tensor(0)

In [11]:
result.logits

tensor([[ 4.1303,  2.3340, -0.2313, -2.8576, -2.6756]],
       grad_fn=<AddmmBackward0>)

In [12]:
int(torch.argmax(result.logits))+1

1

In [13]:
#trying on another example for positve text
tokens = tokenizer.encode('This is amazing, I loved it.', return_tensors = 'pt')
result = model(tokens)
torch.argmax(result.logits)
result.logits
int(torch.argmax(result.logits))+1

5

### 4. Collect Reviews

In [14]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [15]:
reviews

["Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. 

### 5. Load Reviews into Dataframe and score

In [16]:
df = pd.DataFrame(np.array(reviews),columns= ['review'])

In [17]:
df.head()

,review
0,Very cute coffee shop and restaurant. They hav...
1,Six of us met here for breakfast before our wa...
2,"Great service, lovely location, and really ama..."
3,Great place with delicious food and friendly s...
4,Some of the best Milkshakes me and my daughter...


In [18]:
def sentiment_score(text):
    tokens = tokenizer.encode(text, return_tensors = 'pt')
    result = model(tokens)
    torch.argmax(result.logits)
    result.logits
    return int(torch.argmax(result.logits))+1
    

In [19]:
df['score'] = df['review'].apply(lambda x : sentiment_score(x))

In [20]:
df

,review,score
0,Very cute coffee shop and restaurant. They hav...,5
1,Six of us met here for breakfast before our wa...,4
2,"Great service, lovely location, and really ama...",5
3,Great place with delicious food and friendly s...,5
4,Some of the best Milkshakes me and my daughter...,5
5,Great food amazing coffee and tea. Short walk ...,5
6,Ricotta hot cakes! These were so yummy. I ate ...,5
7,It was ok. Had coffee with my friends. I'm new...,3
8,We came for brunch twice in our week-long visi...,5
9,I came to Social brew cafe for brunch while ex...,5


**This was just a basic Sentiment Analaysis Using HuggingFace Model along with few line of WebScrappig using BeautifulSoup**

**Follow for more Such demos and Projects**